# [CDAF] Atividade 5

## Nome: Thiago Pádua de Carvalho

## Matrícula: 2020007066

## Objetivos
- Nessa atividade, estou entregando a pipeline inteira do VAEP implementada para os dados do Wyscout das Top 5 ligas.
- Para cada subtítulo abaixo, vocês devem explicar o que foi feito e à qual seção/subseção/equação do paper "Actions Speak Louder than Goals: Valuing Actions by Estimating Probabilities" ela corresponde. Justifique suas respostas.
- Além disso, após algumas partes do código haverão perguntas que vocês devem responder, possivelmente explorando minimamente o que já está pronto.
- Por fim, vocês devem montar um diagrama do fluxo de funções/tarefas de toda a pipeline do VAEP abaixo. Esse diagrama deve ser enviado como arquivo na submissão do Moodle, para além deste notebook.

## Referências
- [1] https://tomdecroos.github.io/reports/kdd19_tomd.pdf
- [2] https://socceraction.readthedocs.io/en/latest/api/vaep.html

### Carregando os dados

In [2]:
import numpy as np
import pandas as pd

In [3]:
def load_matches(path):
    matches = pd.read_json(path_or_buf=path)
    # as informações dos times de cada partida estão em um dicionário dentro da coluna 'teamsData', então vamos separar essas informações
    team_matches = []
    for i in range(len(matches)):
        match = pd.DataFrame(matches.loc[i, 'teamsData']).T
        match['matchId'] = matches.loc[i, 'wyId']
        team_matches.append(match)
    team_matches = pd.concat(team_matches).reset_index(drop=True)

    return team_matches

In [4]:
def load_players(path):
    players = pd.read_json(path_or_buf=path)
    players['player_name'] = players['firstName'] + ' ' + players['lastName']
    players = players[['wyId', 'player_name']].rename(columns={'wyId': 'player_id'})

    return players

In [5]:
def load_events(path):
    events = pd.read_json(path_or_buf=path)
    # pré processamento em colunas da tabela de eventos para facilitar a conversão p/ SPADL
    events = events.rename(columns={
        'id': 'event_id',
        'eventId': 'type_id',
        'subEventId': 'subtype_id',
        'teamId': 'team_id',
        'playerId': 'player_id',
        'matchId': 'game_id'
    })
    events['milliseconds'] = events['eventSec'] * 1000
    events['period_id'] = events['matchPeriod'].replace({'1H': 1, '2H': 2})

    return events

In [6]:
def load_minutes_played_per_game(path):
    minutes = pd.read_json(path_or_buf=path)
    minutes = minutes.rename(columns={
        'playerId': 'player_id',
        'matchId': 'game_id',
        'teamId': 'team_id',
        'minutesPlayed': 'minutes_played'
    })
    minutes = minutes.drop(['shortName', 'teamName', 'red_card'], axis=1)

    return minutes

In [7]:
leagues = ['England', 'Spain']
events = {}
matches = {}
minutes = {}
for league in leagues:
    path = f'../data/matches/matches_{league}.json'
    matches[league] = load_matches(path)
    path = f'../data/events/events_{league}.json'
    events[league] = load_events(path)
    path = f'../data/minutes_played/minutes_played_per_game_{league}.json'
    minutes[league] = load_minutes_played_per_game(path)

In [8]:
path = f'../data/players.json'
players = load_players(path)
players['player_name'] = players['player_name'].str.decode('unicode-escape')

#### O que foi feito
Primeiramente foi feito o carregamento de uma série de dados: jogos da La Liga e Premier League, jogadores, eventos e minutos jogados por jogo. Houve a leitura dos arquivos de dados em json e transformação em dataframes pandas, seguido de um pré processamento para facilitar a conversão em SPADL.

Essa seção diz respeito a parte de leitura dos dados e pré processamento, descrito na seção 2.1, a qual trata dos desafios impostos pelos dados de event stream.

### SPADL

In [9]:
from tqdm import tqdm
import socceraction.spadl as spd

In [10]:
def spadl_transform(events, matches):
    spadl = []
    game_ids = events.game_id.unique().tolist()
    for g in tqdm(game_ids):
        match_events = events.loc[events.game_id == g]
        match_home_id = matches.loc[(matches.matchId == g) & (matches.side == 'home'), 'teamId'].values[0]
        match_actions = spd.wyscout.convert_to_actions(events=match_events, home_team_id=match_home_id)
        match_actions = spd.play_left_to_right(actions=match_actions, home_team_id=match_home_id)
        match_actions = spd.add_names(match_actions)
        spadl.append(match_actions)
    spadl = pd.concat(spadl).reset_index(drop=True)

    return spadl

In [11]:
spadl = {}
for league in leagues:
    spadl[league] = spadl_transform(events=events[league], matches=matches[league])

100%|██████████| 380/380 [03:04<00:00,  2.06it/s]


### O que foi feito
Após o prcessamento inicial dos dados, seguimos para a conversão dos mesmos para o formato SPADL. Para isso, foi criada uma função que recebe os dataframes de jogos e eventos, retornando um dataframe SPADL. Em seguidas chamamos a função dentro de um for para obtermos as ligas desejadas dentro de um dicionário spadl.

Há aqui uma relação com a seção 2.2, que define e trata da conversão dos dados para o formato SPADL.

### Features

In [12]:
from socceraction.vaep import features as ft

In [13]:
def features_transform(spadl):
    spadl.loc[spadl.result_id.isin([2, 3]), ['result_id']] = 0
    spadl.loc[spadl.result_name.isin(['offside', 'owngoal']), ['result_name']] = 'fail'

    xfns = [
        ft.actiontype_onehot,
        ft.bodypart_onehot,
        ft.result_onehot,
        ft.goalscore,
        ft.startlocation,
        ft.endlocation,
        ft.team,
        ft.time,
        ft.time_delta
    ]

    features = []
    for game in tqdm(np.unique(spadl.game_id).tolist()):
        match_actions = spadl.loc[spadl.game_id == game].reset_index(drop=True)
        match_states = ft.gamestates(actions=match_actions)
        match_feats = pd.concat([fn(match_states) for fn in xfns], axis=1)
        features.append(match_feats)
    features = pd.concat(features).reset_index(drop=True)

    return features

### 1 - O que a primeira e a segunda linhas da função acima fazem? Qual sua hipótese sobre intuito dessas transformações? Como você acha que isso pode impactar o modelo final?

A primeira e segunda linha do programa acima são filtros do dataframe spadl passados como parâmetro. \
Ambas tratam de remover as classificações de offside e owngoal do dataframe, tornando-as fails. A primeira troca 'result-id's de 2 e 3 para 0 e a segunda troca 'offside' e 'owngoal' por fail. 

Minha hipótese é que essas alterações foram feitas com o intuito de refinar os dados para a aplicação do modelo, uma vez que as classificações removidas não são tão relevantes para a análise de valor de ação e ambas podem ser abstraídas para falhas sem alterar negativamente o resultado final. 

O impacto no modelo final será a remoção de dados que não são relevantes para a análise de valor de ação, o que pode tornar o modelo mais preciso. 


In [14]:
features = {}
for league in ['England', 'Spain']:
    features[league] = features_transform(spadl[league])

100%|██████████| 380/380 [00:13<00:00, 27.99it/s]


#### O que foi feito
Definiu-se aqui a função responsável por obter as features que serão inseridas no modelo baseada no dataframe em formato SPADL. features_transform faz um tratamento dos dados, tornando todos os tipos de resultados de ação binários e se utiliza da técnica de one-hot encoding para gerar colunas no dataframe resultante para cada jogo.

O processo está descrito na seção 4.1, que trata da definição das features.

### Labels

In [15]:
import socceraction.vaep.labels as lab

In [16]:
def labels_transform(spadl):
    yfns = [lab.scores, lab.concedes]

    labels = []
    for game in tqdm(np.unique(spadl.game_id).tolist()):
        match_actions = spadl.loc[spadl.game_id == game].reset_index(drop=True)
        labels.append(pd.concat([fn(actions=match_actions) for fn in yfns], axis=1))

    labels = pd.concat(labels).reset_index(drop=True)

    return labels

In [17]:
labels = {}
for league in ['England', 'Spain']:
    labels[league] = labels_transform(spadl[league])

100%|██████████| 380/380 [00:18<00:00, 21.10it/s]


In [18]:
labels['England']['scores'].sum()

7553

In [19]:
labels['England']['concedes'].sum()

2313

### 2- Explique o porquê da quantidade de labels positivos do tipo scores ser muito maior que do concedes. Como você acha que isso pode impactar o modelo final?
Intuitivamente, podemos entender que a quantidade de labels positivos do tipo scores é maior do que a do tipo concedes porque as jogadas ofensivas são melhores construídas, isto é, necessitam de um maior número de estados para se concretizar em gol, enquanto que ações do time com a posse se tornarem gols concedidos podem vir de um número muito menor de estados, como um simples passe errado entre os zagueiros. Podemos ainda afirmar que é menos frequente que essas ações, ditas erros, aconteçam do que investidas de ataque bem sucedidas.

Isso pode impactar o modelo final porque a coluna de labels positivos de concedes é muito esparsa, o que pode levar a um modelo com baixa acurácia. Por outro lado, é esperado que o modelo para scores tenha um desempenho melhor, uma vez que conta com mais dados disponíveis.

#### O que foi feito
Aqui é tratado o problema descrito na seção 4.1 de obter labels para cada ação do jogo baseada no estado corrente. "lab.scores", assinala o valor 1 para ações que resultam em gol para o time com a posse em até k futuras ações - nesse caso K=10 - e 0 para as demais. "lab.concedes" faz o contrário, assinalando 1 para ações que resultam em tomar um gol e 0 para as demais.

### Training Model

In [20]:
import xgboost as xgb
import sklearn.metrics as mt

In [21]:
def train_vaep(X_train, y_train, X_test, y_test):
    models = {}
    for m in ['scores', 'concedes']:
        models[m] = xgb.XGBClassifier(random_state=0, n_estimators=50, max_depth=3)

        print('training ' + m + ' model')
        models[m].fit(X_train, y_train[m])

        p = sum(y_train[m]) / len(y_train[m])
        base = [p] * len(y_train[m])
        y_train_pred = models[m].predict_proba(X_train)[:, 1]
        train_brier = mt.brier_score_loss(y_train[m], y_train_pred) / mt.brier_score_loss(y_train[m], base)
        print(m + ' Train NBS: ' + str(train_brier))
        print()

        p = sum(y_test[m]) / len(y_test[m])
        base = [p] * len(y_test[m])
        y_test_pred = models[m].predict_proba(X_test)[:, 1]
        test_brier = mt.brier_score_loss(y_test[m], y_test_pred) / mt.brier_score_loss(y_test[m], base)
        print(m + ' Test NBS: ' + str(test_brier))
        print()

        print('----------------------------------------')

    return models

In [22]:
models = train_vaep(X_train=features['England'], y_train=labels['England'], X_test=features['Spain'], y_test=labels['Spain'])

training scores model
scores Train NBS: 0.8452154331687597

scores Test NBS: 0.850366923253325

----------------------------------------
training concedes model
concedes Train NBS: 0.964463215550682

concedes Test NBS: 0.9745272575372074

----------------------------------------


### 3 - Por que treinamos dois modelos diferentes? Por que a performance dos dois é diferente?
Treinamos dois modelos diferentes para maximizar a performance de cada um deles. Isso porque tratamos aqui de dois problemas distintos: o de prever gols marcados e o de prever gols sofridos pelo time com a posse de bola. Estas são duas categorias diferentes, cada uma com suas características próprias e, portanto, devem ser analisadas separadamente.

A performance dois dois é diferente porque, como observado anteriormente, há um número maior de labels scores do que de labels concedes. Isso faz com que o modelo de scores tenha uma performance melhor do que o de concedes, uma vez que há mais dados significativos para treiná-lo.

#### O que foi feito
Aqui treinamos os modelos para prever gols marcados e gols sofridos pelo time com a posse de bola. Para isso, utilizamos o modelo XGBoost, que é um modelo de árvore de decisão que utiliza o método de boosting para melhorar a performance. O modelo é treinado com as features e labels obtidas anteriormente. O método aplicado está de acordo com a seção 4 do artigo, a qual indica que "our task simplifies to two separate binary probabilistic classification problems with identical inputs but different labels".

### Predictions

In [23]:
def generate_predictions(features, models):
    preds = {}
    for m in ['scores', 'concedes']:
        preds[m] = models[m].predict_proba(features)[:, 1]
    preds = pd.DataFrame(preds)

    return preds

In [24]:
preds = {}
preds['Spain'] = generate_predictions(features=features['Spain'], models=models)

#### O que foi feito
Aqui utilizamos os modelos treinados anteriormente para fazer previsões de gols marcados e gols sofridos pelo time com a posse de bola a partir das features da liga espanhola. Para isso, utilizamos o método predict_proba do XGBoost, que retorna a probabilidade de cada ação resultar em gol(feito ou concedido), nesse caso.
Essas probabilidades estão de acordo com a definição de VAEP na seção 3.1 e aqui podemos obter seu valor a partir da subtração da coluna de scores pela de concedes. 

### Action Values

In [26]:
import socceraction.vaep.formula as fm

In [27]:
def calculate_action_values(spadl, predictions):
    action_values = fm.value(actions=spadl, Pscores=predictions['scores'], Pconcedes=predictions['concedes'])
    action_values = pd.concat([
        spadl[['original_event_id', 'action_id', 'game_id', 'start_x', 'start_y', 'end_x', 'end_y', 'type_name', 'result_name', 'player_id']],
        predictions.rename(columns={'scores': 'Pscores', 'concedes': 'Pconcedes'}),
        action_values
    ], axis=1)

    return action_values

In [28]:
action_values = {}
action_values['Spain'] = calculate_action_values(spadl=spadl['Spain'], predictions=preds['Spain'])

In [50]:
action_values['Spain']

,original_event_id,action_id,game_id,start_x,start_y,end_x,end_y,type_name,result_name,player_id,Pscores,Pconcedes,offensive_value,defensive_value,vaep_value
0,180864419,0,2565548,38.85,26.52,52.50,34.00,pass,success,3542,0.004560,0.000367,0.000000,-0.000000,0.000000
1,180864418,1,2565548,52.50,34.00,47.25,47.60,pass,success,274435,0.003573,0.000347,-0.000987,0.000020,-0.000967
2,180864420,2,2565548,47.25,47.60,39.90,59.84,pass,success,364860,0.002895,0.000345,-0.000678,0.000002,-0.000676
3,180864421,3,2565548,39.90,59.84,33.60,21.08,pass,success,3534,0.002162,0.000318,-0.000733,0.000027,-0.000706
4,180864422,4,2565548,33.60,21.08,32.55,42.84,pass,success,3695,0.002424,0.001799,0.000262,-0.001481,-0.001219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473889,253302671,1482,2565927,69.30,51.00,92.40,66.64,pass,success,20623,0.033276,0.002812,0.017300,-0.000799,0.016501
473890,253302673,1483,2565927,92.40,66.64,101.85,53.72,dribble,success,122832,0.041886,0.002787,0.008610,0.000025,0.008635
473891,253302674,1484,2565927,101.85,53.72,96.60,50.32,cross,fail,122832,0.017484,0.004722,-0.024402,-0.001935,-0.026337
473892,253302698,1485,2565927,8.40,17.68,9.45,29.92,pass,success,40756,0.007541,0.012254,0.002820,0.005230,0.008050


In [54]:
media_succ = np.mean(action_values['Spain'].loc[(action_values['Spain'].result_name == 'success')
                                                 & (action_values['Spain'].type_name == "shot")].Pscores)
media_fail = np.mean(action_values['Spain'].loc[(action_values['Spain'].result_name == 'fail')
                                                 & (action_values['Spain'].type_name == "shot")].Pscores)

print(f"Média de Pscores para chutes que não resultaram em gol: {media_fail} \n"
      f"Média de Pscores para chutes que resultaram em gol: {media_succ}")

Média de Pscores para chutes que não resultaram em gol: 0.026126321405172348 
Média de Pscores para chutes que resultaram em gol: 0.9823907613754272


### 4 - Explore as ações com Pscores >= 0.95. Por que elas tem um valor tão alto? As compare com ações do mesmo tipo e resultado opostado. Será que o modelo aprende que essa combinação de tipo de ação e resultado está diretamente relacionado à variável y que estamos tentando prever?
Elas tem um valor tão alto porque são chutes que possuem resultado de sucesso, isto é, gols. As ações de chute com resultado oposto têm valor muito próximo de 0, como evidenciado pelas média calculadas acima.

O modelo entende essa diferença em Pscores para chutes com resultado de sucesso e chutes com resultado de falha, uma vez que a diferença entre os valores de Pscores para esses dois tipos de ação é muito grande. Isso mostra que ele aprende que essa combinação de tipo de ação e resultado está diretamente relacionado à variável y que estamos tentando prever.

### 5 - Qual formula do paper corresponde à coluna 'offensive_value' do dataframe action_values? E a coluna 'defensive_value'?

#### O que foi feito
Aqui geramos um dataframe de action values, o qual reúne as métricas de Pscores, Pconcedes vaep_value com dados de ações propriamente ditas, como coordenadas de início e fim, jogador que realizou a ação, etc. \
O dataframe retornado sumariza os dados principais e cáculo de VAEP para cada ação, como descrito no paper. Ele dá uma noção geral dos itens e materializa o conhecimento visto até a seção 4.

### Player Ratings

In [29]:
def calculate_minutes_per_season(minutes_per_game):
    minutes_per_season = minutes_per_game.groupby('player_id', as_index=False)['minutes_played'].sum()

    return minutes_per_season

In [30]:
minutes_per_season = {}
minutes_per_season['Spain'] = calculate_minutes_per_season(minutes['Spain'])

In [31]:
def calculate_player_ratings(action_values, minutes_per_season, players):
    player_ratings = action_values.groupby(by='player_id', as_index=False).agg({'vaep_value': 'sum'}).rename(columns={'vaep_value': 'vaep_total'})
    player_ratings = player_ratings.merge(minutes_per_season, on=['player_id'], how='left')
    player_ratings['vaep_p90'] = player_ratings['vaep_total'] / player_ratings['minutes_played'] * 90
    player_ratings = player_ratings[player_ratings['minutes_played'] >= 600].sort_values(by='vaep_p90', ascending=False).reset_index(drop=True)
    player_ratings = player_ratings.merge(players, on=['player_id'], how='left')
    player_ratings = player_ratings[['player_id', 'player_name', 'minutes_played', 'vaep_total', 'vaep_p90']]

    return player_ratings

In [32]:
player_ratings = {}
player_ratings['Spain'] = calculate_player_ratings(action_values=action_values['Spain'], minutes_per_season=minutes_per_season['Spain'], players=players)

### 6 - Acha que o Top 5 da lista é bem representativo? Compare esse ranqueamento do VAEP com o do xT da Atividade 4. Qual você acha que é mais representativo?

#### O que foi feito